<a href="https://colab.research.google.com/github/muniappabalaji/TimeSeries/blob/main/Electric_Production_TimeSeries_October2025_GL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Approach to the Electric Production Dataset**
1. Load the dataset into Python
2. Audit the dataset (info, first few rows)
3. Plot the dataset
4. Change Date Variable into Index
5. Decompose the time series (Trend, Seasonal and Error/Residual)
6. Moving Average and Exponential Smoothing
   1. Simple Moving Average
   2. Exponential Smoothing
   3. Holts Model (Double Exponential Smoothing)
   4. Holt and Winters Model (Triple Exponential Smoothing)
7. Compare all the MA and ES models and append the results
8. Check for Auto-Correlation
9. Check for Stationarity
10. Application of ARIMA and SARIMA
11. Append the model results
12. Productionize the model for inference

In [ ]:
import pandas as pd
import numpy as np
import sys
import warnings
import itertools
warnings.filterwarnings('ignore')
import statsmodels.api as sm
import statsmodels.tsa.api as smf
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import calendar
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing, Holt